# EQFE Amplification Dynamics: A(φ,t) vs. t for Various C(τ) Shapes

This notebook demonstrates how different environmental correlation functions C(τ) lead to different amplification dynamics A(φ,t) in the Environmental Quantum Field Effects (EQFE) framework.

We explore three key correlation function shapes:
1. **Exponential Decay**: C(τ) = C₀ exp(-|τ|/τc) - Markovian-like environment
2. **Power Law**: C(τ) = C₀ / (1 + |τ|/τc)^α - Non-Markovian with long memory
3. **Oscillatory**: C(τ) = C₀ exp(-|τ|/τc) cos(Ω τ) - Structured spectral density

**Authors**: Justin Todd (justin@pelicansperspective.com), Pelicans Perspective  
**Theory**: Environmental Quantum Field Effects Framework

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

# EQFE Parameters
g = 0.1      # System-environment coupling strength
phi = 0.5    # System phase parameter
T = 1.0      # Temperature (in units where kB = 1)
omega_s = 1.0  # System frequency

In [ ]:
def exponential_correlation(tau, tau_c, C0=1.0):
    """Exponential decay correlation function: C(τ) = C₀ exp(-|τ|/τc)"""
    return C0 * np.exp(-np.abs(tau) / tau_c)

def power_law_correlation(tau, tau_c, alpha=2.0, C0=1.0):
    """Power law correlation function: C(τ) = C₀ / (1 + |τ|/τc)^α"""
    return C0 / (1 + np.abs(tau) / tau_c)**alpha

def oscillatory_correlation(tau, tau_c, Omega, C0=1.0):
    """Oscillatory correlation function: C(τ) = C₀ exp(-|τ|/τc) cos(Ω τ)"""
    return C0 * np.exp(-np.abs(tau) / tau_c) * np.cos(Omega * tau)

def spectral_density(omega, correlation_func, tau_c, **kwargs):
    """Compute spectral density J(ω) via Fourier transform of C(τ)"""
    def integrand(tau):
        return correlation_func(tau, tau_c, **kwargs) * np.cos(omega * tau)
    
    result, _ = integrate.quad(integrand, 0, np.inf)
    return 2 * result  # Factor of 2 from symmetry

In [ ]:
def compute_amplification(t, phi, g, correlation_func, tau_c, **kwargs):
    """
    Compute EQFE amplification A(φ,t) using perturbative expansion.
    
    Based on the theoretical framework:
    A(φ,t) ≈ 1 + g²∫₀ᵗ ∫₀ᵗ C(τ₁-τ₂) K(τ₁,τ₂,φ) dτ₁dτ₂
    
    where K(τ₁,τ₂,φ) encodes the system-environment coupling dynamics.
    """
    def kernel(tau1, tau2, phi):
        # Simplified kernel based on system dynamics
        delta_tau = tau1 - tau2
        return np.cos(phi * delta_tau) * np.exp(-0.1 * (tau1 + tau2))
    
    def integrand(tau1, tau2):
        correlation = correlation_func(tau1 - tau2, tau_c, **kwargs)
        return correlation * kernel(tau1, tau2, phi)
    
    # Double integration over time
    result = 0.0
    dt = t / 100  # Integration step
    for i in range(100):
        tau1 = i * dt
        for j in range(100):
            tau2 = j * dt
            if tau1 <= t and tau2 <= t:
                result += integrand(tau1, tau2) * dt * dt
    
    return 1 + g**2 * result

In [ ]:
# Time array
t_max = 10.0
t_array = np.linspace(0.01, t_max, 100)

# Correlation time scales
tau_c_values = [0.5, 1.0, 2.0]

# Initialize figure
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('EQFE Amplification Dynamics: A(φ,t) vs. t for Different C(τ) Shapes', fontsize=16)

# Plot 1: Exponential decay correlation
ax = axes[0, 0]
for tau_c in tau_c_values:
    A_values = [compute_amplification(t, phi, g, exponential_correlation, tau_c) for t in t_array]
    ax.plot(t_array, A_values, label=f'τc = {tau_c}', linewidth=2)

ax.set_xlabel('Time t')
ax.set_ylabel('Amplification A(φ,t)')
ax.set_title('Exponential Decay: C(τ) = exp(-|τ|/τc)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Power law correlation
ax = axes[0, 1]
for tau_c in tau_c_values:
    A_values = [compute_amplification(t, phi, g, power_law_correlation, tau_c, alpha=2.0) for t in t_array]
    ax.plot(t_array, A_values, label=f'τc = {tau_c}', linewidth=2)

ax.set_xlabel('Time t')
ax.set_ylabel('Amplification A(φ,t)')
ax.set_title('Power Law: C(τ) = 1/(1 + |τ|/τc)²')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Oscillatory correlation
ax = axes[1, 0]
Omega = 2.0  # Oscillation frequency
for tau_c in tau_c_values:
    A_values = [compute_amplification(t, phi, g, oscillatory_correlation, tau_c, Omega=Omega) for t in t_array]
    ax.plot(t_array, A_values, label=f'τc = {tau_c}', linewidth=2)

ax.set_xlabel('Time t')
ax.set_ylabel('Amplification A(φ,t)')
ax.set_title(f'Oscillatory: C(τ) = exp(-|τ|/τc)cos(Ωτ), Ω = {Omega}')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Comparison at fixed τc
ax = axes[1, 1]
tau_c_fixed = 1.0
A_exp = [compute_amplification(t, phi, g, exponential_correlation, tau_c_fixed) for t in t_array]
A_pow = [compute_amplification(t, phi, g, power_law_correlation, tau_c_fixed, alpha=2.0) for t in t_array]
A_osc = [compute_amplification(t, phi, g, oscillatory_correlation, tau_c_fixed, Omega=2.0) for t in t_array]

ax.plot(t_array, A_exp, label='Exponential', linewidth=2)
ax.plot(t_array, A_pow, label='Power Law', linewidth=2)
ax.plot(t_array, A_osc, label='Oscillatory', linewidth=2)

ax.set_xlabel('Time t')
ax.set_ylabel('Amplification A(φ,t)')
ax.set_title(f'Comparison at τc = {tau_c_fixed}')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analysis and Key Insights

### 1. **Exponential Decay (Markovian-like)**
- **Short memory**: Environment correlation decays exponentially
- **Amplification**: Moderate enhancement with gradual approach to steady state
- **Physical interpretation**: Classical thermal environment with short correlation time

### 2. **Power Law (Non-Markovian)**
- **Long memory**: Algebraic decay preserves correlations longer
- **Amplification**: Enhanced performance due to persistent memory effects
- **Physical interpretation**: Structured environments with long-range correlations

### 3. **Oscillatory (Structured Spectral Density)**
- **Coherent dynamics**: Oscillations in correlation function
- **Amplification**: Periodic enhancement matching environmental frequency
- **Physical interpretation**: Cavity-like environments or phonon modes

### Key EQFE Predictions:
1. **Memory Advantage**: Non-Markovian environments (power law) show superior amplification
2. **Resonance Effects**: Oscillatory environments create periodic enhancement windows
3. **Time Scale Matching**: Optimal performance when system and environment time scales align

### Experimental Implications:
- Biological systems may exploit power-law environments for enhanced quantum coherence
- Structured environments (proteins, membranes) provide oscillatory correlation patterns
- Temperature and coupling strength determine the transition between regimes